## 实现支持向量机
- 使用**SMO算法**训练svm模型
- 线性可分支持向量机（硬间隔）
- 线性支持向量机（软间隔）
- 非线性支持向量机（核函数）
- 其中**线性可分支持向量机**可看做**线性支持向量机**的特殊形式，而**线性支持向量机**也可看做**非线性支持向量机**的特殊形式

In [1]:
import numpy as np

In [63]:
class Kernel(object):
    def __init__(self):
        pass
    def calKernel(self, x, y, sigma):
        pass
class LinearKernel(Kernel):
    """线性核函数类，主要实现两个向量（一维向量）的内积
    Args:
        x,(numpy.narray)
        y,(numpy.narray)
    """
    def __init__(self):
        pass
    def calKernel(self, x, y, sigma=None):
        if len(x.shape) > 2 or len(y.shape) > 2:
            raise AssertionError(r"输入向量的维度超过了2，无法转换为1维向量")
        if len(x.shape) == 2:
            x = x.squeeze()
        if len(y.shape) == 2:
            y = y.squeeze()
        assert x.shape == y.shape
        # 计算内积
        return np.dot(x, y)

class RbfKernel(object):
    """高斯径向基核函数类，
    Args:
        x,(numpy.narray)
        y,(numpy.narray)
        sigma, int
    """
    def __init__(self):
        pass
    def calKernel(self, x, y, sigma=1):
        if len(x.shape) > 2 or len(y.shape) > 2:
            raise AssertionError(r"输入向量的维度超过了2，无法转换为1维向量")
        if len(x.shape) == 2:
            x = x.squeeze()
        if len(y.shape) == 2:
            y = y.squeeze()
        assert x.shape == y.shape
        # 径向基
        diff = x - y
        return np.exp(-(np.dot(diff, diff))/(2*sigma**2))

In [110]:
lkernel = RbfKernel()
x = np.array([4, 5])
y = np.array([-2, 7])
print lkernel.calKernel(x, y)

2.06115362244e-09


In [116]:
class SVM(object):
    """实现非线性支持向量机模型的训练、预测
    Args:
        kernel,(str)，决定使用哪一种核函数
        C,(float),惩罚参数
        epsilon(flaot),松弛变量
    """
    def __init__(self, kernel='gaussian', C=10.0, epsilon=0.001, sigma=10):
        if kernel == 'linear':
            self.kernel = LinearKernel()
        else:
            self.kernel = RbfKernel()
        self.C = C
        self.epsilon = epsilon
        self.sigma = sigma
        self.W = None
        self.b = 0.0
        self.alpha = None
        self.train_x = None
        self.train_y = None
        self.kernel_value = None
        self.m = 0
        self.n = 0
        self.support_vectors = []
    def __getKernel(self):
        self.kernel_value = np.zeros(shape=(self.m, self.m))
        for i in range(self.m):
            for j in range(self.m):
                if j < i:
                    self.kernel_value[i][j] = self.kernel_value[j][i]
                else:
                    self.kernel_value[i][j] = self.kernel.calKernel(self.train_x[i], 
                                                                    self.train_x[j], 
                                                                    self.sigma)
    def __predict(self, index=None):
        if index is None:
            pre_y = np.empty_like(self.train_y)
            # 循环解法
    #         for i in range(self.m):
    #             pre_y[i] = np.dot(self.alpha * self.train_y, self.kernel_value[i]) + self.b
            # 向量化解法
            pre_y = np.dot(self.kernel_value, (self.alpha * 
                                               self.train_y).reshape(-1, 1)
                          ).squeeze() + self.b
        else:
            assert index < self.m
            pre_y = np.dot(self.alpha * self.train_y, self.kernel_value[index]) + self.b
        return pre_y
    def fit(self, train_x, train_y):
        self.train(train_x, train_y)
    def train(self, train_x, train_y):
        self.train_x = train_x
        self.train_y = train_y
        self.m, self.n = self.train_x.shape
        # 根据当前阶段的alpha,计算当前的预测值
        self.alpha = np.zeros(self.m)
        self.__getKernel()
        pre_y = self.__predict()
        E = pre_y - self.train_y
        pre_y_plus_train_y = pre_y * self.train_y
        # 按照启发式算法寻找alpha_x与alpha_y
        while True:
            is_satisfied = True
            cur_alpha_x = -1
            cur_bias = 0
            for i in range(self.m):
                if self.alpha[i] > 0 and self.alpha[i] < self.C:
                    if pre_y_plus_train_y[i] != 1:
                        if is_satisfied is True :
                            cur_bias = np.abs(1-pre_y_plus_train_y[i])
                            if cur_bias > self.epsilon:
                                is_satisfied = False
                                cur_alpha_x = i
                        elif cur_bias < np.abs(1-pre_y_plus_train_y[i]):
                            cur_alpha_x = i
                            cur_bias = np.abs(1-pre_y_plus_train_y[i])
            if is_satisfied is True: # 说明所有0<alpha_x<C <==> prey_i*trainy_i == 1全部成立，
                # 下一步判断其他两种情况
                for i in range(self.m):
                    if self.alpha[i] == 0:
                        if pre_y_plus_train_y[i] < 1:
                            if is_satisfied is True:
                                cur_bias = np.abs(1-pre_y_plus_train_y[i])
                                if cur_bias > self.epsilon:
                                    is_satisfied = False
                                    cur_alpha_x = i
                            elif cur_bias < np.abs(1-pre_y_plus_train_y[i]):
                                cur_alpha_x = i
                                cur_bias = np.abs(1-pre_y_plus_train_y[i])
                    elif self.alpha[i] == self.C:
                        if pre_y_plus_train_y[i] > 1:
                            if is_satisfied is True:
                                cur_bias = np.abs(1-pre_y_plus_train_y[i])
                                if cur_bias > self.epsilon:
                                    is_satisfied = False
                                    cur_alpha_x = i
                            elif cur_bias < np.abs(1-pre_y_plus_train_y[i]):
                                cur_alpha_x = i
                                cur_bias = np.abs(1-pre_y_plus_train_y[i])
            if is_satisfied is True: # 全部都满足kkt条件，退出循环
                break
            # 成功的找到了alpha_x,即第一个alpha,再次开始遍历alpha,找到alpha_y，即第二个alpha
            # 按照使|E_x - E_y|最大化的启发式方法
            E_max, E_min = np.max(E), np.min(E)
            cur_alpha_y = -1
            if np.abs(E_max - E[cur_alpha_x]) > np.abs(E_min - E[cur_alpha_x]):
                cur_alpha_y = np.argmax(E)
            else:
                cur_alpha_y = np.argmin(E)
            # 找到了alpha_y，下一步更新alpha_x与alpha_y
            L = 0
            H = 0
            if self.train_y[cur_alpha_x] == self.train_y[cur_alpha_y]:
                L = max(0, self.alpha[cur_alpha_x]+self.alpha[cur_alpha_y]-self.C)
                H = min(self.C, self.alpha[cur_alpha_x]+self.alpha[cur_alpha_y])
            else:
                L = max(0, self.alpha[cur_alpha_y]-self.alpha[cur_alpha_x])
                H = min(self.C, self.alpha[cur_alpha_y]-self.alpha[cur_alpha_x]+self.C)
            
            eta = self.kernel_value[cur_alpha_x][cur_alpha_x]+\
            self.kernel_value[cur_alpha_y][cur_alpha_y]-\
            2*self.kernel_value[cur_alpha_x][cur_alpha_y]
            
            next_alpha_y_unc = self.alpha[cur_alpha_y] + \
            self.train_y[cur_alpha_y]*(E[cur_alpha_x]- E[cur_alpha_y])/ eta
            
            next_alpha_y = 0
            if next_alpha_y_unc > H:
                next_alpha_y = H
            elif next_alpha_y_unc >= L:
                next_alpha_y = next_alpha_y_unc
            else:
                next_alpha_y = L
            
            next_alpha_x = self.alpha[cur_alpha_x] + \
            self.train_y[cur_alpha_x]*self.train_y[cur_alpha_y]*\
            (self.alpha[cur_alpha_y] - next_alpha_y)
            
            # 更新b的值
            b_new_1 = -E[cur_alpha_x] - self.train_y[cur_alpha_x]*\
            self.kernel_value[cur_alpha_x][cur_alpha_x]*\
            (next_alpha_x - self.alpha[cur_alpha_x]) - \
            self.train_y[cur_alpha_y]*\
            self.kernel_value[cur_alpha_y][cur_alpha_x]*\
            (next_alpha_y - self.alpha[cur_alpha_y]) + self.b

            b_new_2 = -E[cur_alpha_y] - self.train_y[cur_alpha_x]*\
            self.kernel_value[cur_alpha_x][cur_alpha_y]*\
            (next_alpha_x - self.alpha[cur_alpha_x]) - \
            self.train_y[cur_alpha_y]*\
            self.kernel_value[cur_alpha_y][cur_alpha_y]*\
            (next_alpha_y - self.alpha[cur_alpha_y]) + self.b
            
            self.b = (b_new_1 + b_new_2) / 2
            
            # 更新alpha_x,alpha_y,E_alpha_x,E_alpha_y
            self.alpha[cur_alpha_x] = next_alpha_x
            self.alpha[cur_alpha_y] = next_alpha_y
            pre_y[cur_alpha_x] = self.__predict(index = cur_alpha_x)
            pre_y[cur_alpha_y] = self.__predict(index = cur_alpha_y)
            E[cur_alpha_x] = pre_y[cur_alpha_x] - self.train_y[cur_alpha_x]
            E[cur_alpha_y] = pre_y[cur_alpha_y] - self.train_y[cur_alpha_y]
        # 利用alpha更新w
        self.W = np.dot(self.train_x.T, (self.alpha * self.train_y
                                        ).reshape(-1, 1)).squeeze()
        # 记录支持向量
        for i in range(self.m):
            if self.alpha[i] > 0:
                self.support_vectors.append(i)
    def output(self, x):
        return np.dot(x, self.W.reshape(-1, 1)).squeeze()+self.b
    def predict(self, x):
        return np.sign(self.output(x))

### 简单数据集上的测试

In [117]:
train_x = np.array([[0,0],[0,1],[1,0],[0.5, 0.6],[0.8,1],[1,1]])
train_y = np.array([-1, -1, -1, 1, 1, 1])
test_x = np.array([[0.4, 0.3],[-1, 0.5],[1, 2], [0.6, 0.7]])
test_y = np.array([-1, -1, 1, 1])

In [118]:
svm = SVM(kernel='linear', C=2.0, epsilon=0.001, sigma=10)

In [119]:
svm.fit(train_x, train_y)
print svm.predict(test_x)
print svm.predict(train_x)

[-1. -1.  1.  1.]
[-1.  1.  1.  1.  1.  1.]


In [120]:
print svm.W, svm.b

[ 2.6  3.2] -2.25


In [121]:
print svm.alpha

[ 2.  2.  2.  2.  2.  2.]


In [122]:
print svm.kernel_value

[[ 0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.    0.    0.6   1.    1.  ]
 [ 0.    0.    1.    0.5   0.8   1.  ]
 [ 0.    0.6   0.5   0.61  1.    1.1 ]
 [ 0.    1.    0.8   1.    1.64  1.8 ]
 [ 0.    1.    1.    1.1   1.8   2.  ]]


In [123]:
print svm.support_vectors

[0, 1, 2, 3, 4, 5]


## 在mnist数据集上测试